In [1]:
import requests
import sqlite3
import pandas as pd
import numpy as np
import time
from datetime import datetime

# Extraction

## Open Meteo API Call

https://open-meteo.com/en/docs/historical-forecast-api?location_mode=csv_coordinates&hourly=&daily=temperature_2m_max,temperature_2m_min,precipitation_sum&temperature_unit=fahrenheit

In [2]:
def openMeteo_APICall(latitude, longitude, start_date, end_date):

    url = "https://archive-api.open-meteo.com/v1/archive"

    params = {
        "latitude": latitude,
        "longitude": longitude,
	    "start_date": start_date,
        "end_date": end_date,
        "daily": ["temperature_2m_max", "temperature_2m_min", "precipitation_sum", "rain_sum", "snowfall_sum"],
        "temperature_unit": "fahrenheit",
        "timezone": "America/New_York" #Eastern Time
    }

    response = requests.get(url, params=params, timeout=30)
    data = response.json()
    

    if response.status_code == 200: #OK
        if not "error" in data:
            return data
        else:
            print("Response Error: ", data['error'], data['reason'])
    else:
        print("HTTP Call Error:", response.status_code, response.text)
        return None

In [ ]:
weather_data = openMeteo_APICall(33.8034, -84.3963, "2018-01-01", "2023-12-31")

## NHTSA API By Location Call

https://crashviewer.nhtsa.dot.gov/CrashAPI

In [3]:
def NHTSA_APICall(stateCode, countyCode, startYear, endYear):
    
    url = f"https://crashviewer.nhtsa.dot.gov/CrashAPI/crashes/GetCrashesByLocation?fromCaseYear={startYear}&toCaseYear={endYear}&state={stateCode}&county={countyCode}&format=json"
    
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
    }

    response = requests.get(url, headers=headers, timeout=30)
    data = response.json()
    
    if response.status_code == 200: #OK
        return data
    else:
        print("Error:", response.status_code, response.text)
        return None

In [ ]:
crash_data = NHTSA_APICall(13, 121, 2018, 2023)

## NHTSA API Case Specifics Call

In [ ]:
def NHTSA_GetCaseSpecifics(db, stateCode, year):

    conn = sqlite3.connect(db)

    crash_df = pd.read_sql_query("SELECT * FROM crashes", conn)

    caseSpecDB = pd.DataFrame(columns=["state_case","year","month","day"])
    tempdb = pd.read_sql_query(f"SELECT name FROM sqlite_master WHERE type='table' AND name='case_specifics'", conn)
    
    if len(tempdb["name"]) > 0: #case specifics table already exists in database
        caseSpecDB = pd.read_sql_query("SELECT * FROM case_specifics", conn)

        for scase in crash_df["state_case"]:
            if scase in caseSpecDB["state_case"].unique():
                print(f"Skipping case {scase}, as it is in the dataset already")
            else:
                url = f"https://crashviewer.nhtsa.dot.gov/CrashAPI/crashes/GetCaseDetails?stateCase={scase}&caseYear={year}&state={stateCode}&format=json"

                new_df = NHTSA_CaseSpec_APICall(url, scase)
                caseSpecDB = pd.concat([caseSpecDB, new_df])

        return caseSpecDB
    
def NHTSA_CaseSpec_APICall(url, scase):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
    }

    time.sleep(3) #Being cautious since this will be sending multiple API calls in rapid succession when new crash specifics emerge

    response = requests.get(url, headers=headers, timeout=30)
    data = response.json()

    new_df = pd.DataFrame([[scase, None, None, None]])
    if data["Count"] == 1: #Some cases don't have detailed information released yet, in which case the API will return a count of 0
        year = data["Results"][0][0]['CrashResultSet']['YEAR']
        month = f"{int(data["Results"][0][0]['CrashResultSet']['MONTH']):02}"
        day = f"{int(data["Results"][0][0]['CrashResultSet']['DAY']):02}"

        new_df = pd.DataFrame([[scase, year, month, day]], columns = ["state_case","year","month","day"])

        print(f"Successfully got date for {scase}")

    return new_df


In [ ]:
casespecs_db = NHTSA_GetCaseSpecifics("crashinfo.db", 13, 2023)

In [ ]:
scase = "130112"
url = f"https://crashviewer.nhtsa.dot.gov/CrashAPI/crashes/GetCaseDetails?stateCase={scase}&caseYear=2023&state=13&format=json"

temp_df = NHTSA_CaseSpec_APICall(url, scase)
temp_df

## OpenAQ API Call

In [4]:
from openaq import OpenAQ

def OpenAQ_Location_APICall(latitude, longitude, radius, start_date, end_date):

    client = OpenAQ(api_key="7755a70a98b5f75b8d6c20c291ea8d3bec9a8b18542a10126358c7d19f49a75a")
    response = client.locations.list(
        coordinates = [latitude, longitude],
        radius = radius,
        limit = 1000
    )
    #client.close()

    return response

In [ ]:
latitude = 33.8034
longitude = -84.3963
openAQ = OpenAQ_Location_APICall(latitude, longitude, 10000, "2023-01-01", "2023-01-17")

In [57]:
for loc in openAQ.results:
    print(loc.id, loc.name, loc.coordinates, loc.sensors)

1951 United Ave Coordinates(latitude=33.7206, longitude=-84.3578) [SensorBase(id=3451, name='o3 ppm', parameter=ParameterBase(id=10, name='o3', units='ppm', display_name='O₃')), SensorBase(id=3453, name='pm25 µg/m³', parameter=ParameterBase(id=2, name='pm25', units='µg/m³', display_name='PM2.5')), SensorBase(id=3452, name='so2 ppm', parameter=ParameterBase(id=9, name='so2', units='ppm', display_name='SO₂'))]
1972 NR-Georgia Tech Coordinates(latitude=33.778315, longitude=-84.391418) [SensorBase(id=5077788, name='co ppm', parameter=ParameterBase(id=8, name='co', units='ppm', display_name='CO')), SensorBase(id=5077750, name='no ppm', parameter=ParameterBase(id=35, name='no', units='ppm', display_name='NO')), SensorBase(id=5077563, name='no2 ppm', parameter=ParameterBase(id=7, name='no2', units='ppm', display_name='NO₂')), SensorBase(id=5077742, name='nox ppm', parameter=ParameterBase(id=19840, name='nox', units='ppm', display_name='NOx')), SensorBase(id=5077784, name='pm25 µg/m³', paramet

In [2]:
def OpenAQ_Sensor_APICall(sensorID, limit):
    
    client = OpenAQ(api_key="7755a70a98b5f75b8d6c20c291ea8d3bec9a8b18542a10126358c7d19f49a75a")
    response = client.measurements.list(
        sensors_id=sensorID,
        data="days",
        limit=limit,
        page=3
    )

    return response

In [5]:
sensorData = OpenAQ_Sensor_APICall(1972, 1000) #ga tech ozone sensor

In [7]:
for item in sensorData.results:
    print(item.period.datetime_from.local, item.value, item.summary.min, item.summary.max, item.parameter.name, item.parameter.units)

2021-12-29T00:00:00-06:00 0.0197 0.012000000104308128 0.028999999165534973 o3 ppm
2021-12-30T00:00:00-06:00 0.0162 0.0010000000474974513 0.03500000014901161 o3 ppm
2021-12-31T00:00:00-06:00 0.0211 0.010999999940395355 0.03099999949336052 o3 ppm
2022-01-01T00:00:00-06:00 0.0218 0.01600000075995922 0.027000000700354576 o3 ppm
2022-01-02T00:00:00-06:00 0.0187 0.008999999612569809 0.02800000086426735 o3 ppm
2022-01-03T00:00:00-06:00 0.0292 0.019999999552965164 0.032999999821186066 o3 ppm
2022-01-04T00:00:00-06:00 0.0201 0.009999999776482582 0.028999999165534973 o3 ppm
2022-01-05T00:00:00-06:00 0.0215 0.0020000000949949026 0.039000000804662704 o3 ppm
2022-01-06T00:00:00-06:00 0.0219 0.013000000268220901 0.027000000700354576 o3 ppm
2022-01-07T00:00:00-06:00 0.025 0.014999999664723873 0.03099999949336052 o3 ppm
2022-01-08T00:00:00-06:00 0.0285 0.01899999938905239 0.03700000047683716 o3 ppm
2022-01-09T00:00:00-06:00 0.0254 0.014999999664723873 0.03400000184774399 o3 ppm
2022-01-10T00:00:00-06:

# Transformation

In [9]:
def transform_weather(data):
    weather_df = pd.DataFrame({
                    "latitude": data["latitude"],
                    "longitude": data["longitude"],
                    "date": data["daily"]["time"],
                    "temp_max_F": data["daily"]["temperature_2m_max"],
                    "temp_min_F": data["daily"]["temperature_2m_min"],
                    "precip_sum": data["daily"]["precipitation_sum"],
                    "rain_sum": data["daily"]["rain_sum"],
                    "snowfall_sum": data["daily"]["snowfall_sum"]
                })
    return weather_df

def transform_accidents(data):
    accident_df = pd.DataFrame.from_dict(data['Results'][0])

    accident_df = accident_df.drop(columns=["CITY","COUNTY","STATE","TWAY_ID2","VE_FORMS"])

    accident_df = accident_df.rename(columns={
        "CITYNAME": "city",
        "COUNTYNAME": "county",
        "CaseYear": "year",
        "FATALS": "fatals",
        "LATITUDE": "latitude",
        "LONGITUD": "longitude",
        "STATENAME": "state",
        "ST_CASE": "state_case",
        "TOTALVEHICLES": "vehicles",
        "TWAY_ID": "road_occurred"
    })

    return accident_df

def transform_ozone_measure(data):
    ozone_df = pd.DataFrame(columns=["datetime","mean_values", "minimum_value", "maximum_value", "name", "units"])
    for item in data.results:
        date = item.period.datetime_from.local[:10]

        new_df = pd.DataFrame([[
            date,
            item.value,
            item.summary.min, 
            item.summary.max, 
            item.parameter.name, 
            item.parameter.units
        ]], columns=["datetime","mean_values", "minimum_value", "maximum_value", "name", "units"])

        ozone_df = pd.concat([ozone_df, new_df])
    return ozone_df

In [ ]:
weather_df = transform_weather(weather_data)
crash_df = transform_accidents(crash_data)

In [10]:
ozone_df = transform_ozone_measure(sensorData)
ozone_df

C:\Users\dbeck\AppData\Local\Temp\ipykernel_13384\591620894.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ozone_df = pd.concat([ozone_df, new_df])


,datetime,mean_values,minimum_value,maximum_value,name,units
0,2021-12-29,0.0197,0.012,0.029,o3,ppm
0,2021-12-30,0.0162,0.001,0.035,o3,ppm
0,2021-12-31,0.0211,0.011,0.031,o3,ppm
0,2022-01-01,0.0218,0.016,0.027,o3,ppm
0,2022-01-02,0.0187,0.009,0.028,o3,ppm
...,...,...,...,...,...,...
0,2024-11-13,0.0194,0.016,0.023,o3,ppm
0,2024-11-14,0.0233,0.005,0.042,o3,ppm
0,2024-11-15,0.0225,0.007,0.037,o3,ppm
0,2024-11-16,0.0231,0.006,0.040,o3,ppm


# Loading To SQLite

In [ ]:
def load_to_database(weather_df, crash_df, casespecs_df, ozone_df, db="crashinfo.db"):
    conn = sqlite3.connect(db)

    weather_df.to_sql("weather", conn, if_exists="replace", index=False)
    print("Weather (weather) data loaded into SQLite")
    crash_df.to_sql("crashes", conn, if_exists="replace", index=False)
    print("Crash data (crashes) loaded into SQLite")
    casespecs_df.to_sql("case_specifics", conn, if_exists="replace", index=False)
    print("Case Specific (case_specifics) data loaded into SQLite")
    ozone_df.to_sql("gtech_ozone", conn, if_exists="replace", index=False)
    print("Georgia Tech Ozone (gtech_ozone) data loaded into SQLite")

    conn.close()

In [ ]:
load_to_database(weather_df, crash_df, casespecs_db, db="crashinfo.db")

# Accessing SQLite

In [12]:
conn = sqlite3.connect("crashinfo.db")
cursor = conn.cursor()

In [ ]:
pd.read_sql_query("SELECT * FROM crashes", conn)

In [ ]:
tempdb = pd.read_sql_query(f"SELECT name FROM sqlite_master WHERE type='table' AND name='{"weather"}'", conn)

if len(tempdb["name"]) > 0:
    print("Table exists.")
else:
    print("Table does not exist.")

# Testing

In [ ]:
url = f"https://crashviewer.nhtsa.dot.gov/CrashAPI/crashes/GetCaseDetails?stateCase=130035&caseYear=2023&state=13&format=json"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}

response = requests.get(url, headers=headers, timeout=30)
data = response.json()

In [ ]:
case_specifics = pd.DataFrame([[1,2,3,4]], columns=["state_case","year","month","day"])

new_df = pd.DataFrame([[
    "130035",
    data["Results"][0][0]['CrashResultSet']['YEAR'],
    data["Results"][0][0]['CrashResultSet']['MONTH'],
    data["Results"][0][0]['CrashResultSet']['DAY']
]], columns = ["state_case","year","month","day"])

case_specifics = pd.concat([case_specifics, new_df])

case_specifics


In [ ]:
caseSpecDB = pd.read_sql_query("SELECT * FROM case_specifics", conn)
crashDB = pd.read_sql_query("SELECT * FROM crashes", conn)
for item in crashDB["state_case"]:
    if item not in caseSpecDB["state_case"].unique():
        print(item)

In [44]:
weatherDB = pd.read_sql_query("SELECT * FROM case_specifics", conn)
weatherDB

,state_case,year,month,day
0,130035,2023,02,13
1,130037,2022,01,14
2,130038,2022,01,14
3,130043,2022,01,12
4,130045,2022,01,18
...,...,...,...,...
237,131736,2022,06,08
238,131748,2022,10,26
239,131766,2022,09,30
240,131770,2022,07,22


In [16]:
crashDB = pd.read_sql_query("SELECT * FROM crashes", conn)
crashDB

,city,county,year,fatals,latitude,longitude,state,state_case,vehicles,road_occurred
0,ATLANTA,FULTON (121),2023,1,33.75976667,-84.482205560,Georgia,130035,3,I-20
1,ATLANTA,FULTON (121),2022,1,33.77472500,-84.390136110,Georgia,130037,2,I-75
2,ATLANTA,FULTON (121),2022,1,33.74546389,-84.396327780,Georgia,130038,1,I-20 RAMP
3,NOT APPLICABLE,FULTON (121),2022,1,33.54270278,-84.571522220,Georgia,130043,2,CR-374000/OAKLEY INDUSTRIAL BL
4,COLLEGE PARK,FULTON (121),2022,1,33.61910278,-84.473147220,Georgia,130045,1,SR-279
...,...,...,...,...,...,...,...,...,...,...
243,ATLANTA,FULTON (121),2022,1,33.74471944,-84.399269440,Georgia,131736,2,I-20
244,ATLANTA,FULTON (121),2022,1,33.68130556,-84.415002780,Georgia,131748,3,MU-30963 CLEVELAND AVE
245,Not Reported,FULTON (121),2022,2,33.74320278,-84.377777780,Georgia,131766,2,I-20
246,EAST POINT,FULTON (121),2022,1,33.63746111,-84.492788890,Georgia,131770,4,I-285


In [83]:
tempdb = pd.read_sql_query(f"SELECT name FROM sqlite_master WHERE type='table'", conn)
tempdb

,name
0,weather
1,crashes
2,case_specifics
3,gtech_ozone


In [82]:
ozoneDB = pd.read_sql_query("SELECT * FROM gtech_ozone", conn)
ozoneDB

,datetime,mean_values,minimum_value,maximum_value,name,units
0,2021-12-29,0.0197,0.012,0.029,o3,ppm
1,2021-12-30,0.0162,0.001,0.035,o3,ppm
2,2021-12-31,0.0211,0.011,0.031,o3,ppm
3,2022-01-01,0.0218,0.016,0.027,o3,ppm
4,2022-01-02,0.0187,0.009,0.028,o3,ppm
...,...,...,...,...,...,...
995,2024-11-13,0.0194,0.016,0.023,o3,ppm
996,2024-11-14,0.0233,0.005,0.042,o3,ppm
997,2024-11-15,0.0225,0.007,0.037,o3,ppm
998,2024-11-16,0.0231,0.006,0.040,o3,ppm


In [75]:
crashview = pd.read_sql_query("SELECT name, type FROM sqlite_master WHERE type='view';", conn)
crashview

,name,type
0,crashes_by_precip,view
1,crash_weather,view
2,weather_crashes_by_day,view
3,crashes_precip_freq,view
4,crash_ozone,view
5,ozone_crashes_by_day,view
6,crashes_ozone_freq,view


In [76]:
crash_precip = pd.read_sql_query("SELECT * FROM crashes_precip_freq", conn)
crash_precip

,precipitation_buckets,crash_count,total_days,crash_freq,avg_fatals,avg_vehicles_involved
0,>20mm,10,24,0.416667,1.100000,1.700000
1,0mm,52,169,0.307692,1.115385,1.538462
2,2.5mm-10mm,13,45,0.288889,1.076923,1.384615
3,10mm-20mm,13,45,0.288889,1.000000,1.461538
4,0mm-2.5mm,20,103,0.194175,1.200000,1.500000


In [77]:
crash_weather = pd.read_sql_query("SELECT MIN(mean_values), MAX(mean_values) FROM gtech_ozone", conn)
crash_weather

,MIN(mean_values),MAX(mean_values)
0,0.002,0.0516


In [78]:
crash_weather = pd.read_sql_query("SELECT * FROM crash_ozone LIMIT 10", conn)
crash_weather

,state_case,datetime,city,county,state,fatals,road_occurred,vehicles,mean_ozone_ppm,min_ozone_ppm,max_ozone_ppm
0,130035,2023-02-13,ATLANTA,FULTON (121),Georgia,1,I-20,3,0.0046,0.004,0.006
1,130037,2022-01-14,ATLANTA,FULTON (121),Georgia,1,I-75,2,0.0277,0.001,0.040
2,130038,2022-01-14,ATLANTA,FULTON (121),Georgia,1,I-20 RAMP,1,0.0277,0.001,0.040
3,130043,2022-01-12,NOT APPLICABLE,FULTON (121),Georgia,1,CR-374000/OAKLEY INDUSTRIAL BL,2,0.0145,0.000,0.041
4,130045,2022-01-18,COLLEGE PARK,FULTON (121),Georgia,1,SR-279,1,0.0223,0.000,0.037
5,130065,2022-02-06,SANDY SPRINGS,FULTON (121),Georgia,2,SR-400,2,0.0205,0.000,0.048
6,130074,2023-03-18,ATLANTA,FULTON (121),Georgia,1,MU-304005 GODBY RD,2,0.0438,0.042,0.046
7,130076,2023-03-19,ATLANTA,FULTON (121),Georgia,1,SR-140 LEE ST,4,0.0445,0.036,0.049
8,130077,2023-03-20,ATLANTA,FULTON (121),Georgia,2,SR-166 CAMPBELLTON RD,2,0.0242,0.003,0.045
9,130081,2022-02-02,ATLANTA,FULTON (121),Georgia,1,I-20,1,0.0315,0.011,0.046


In [ ]:
testquery = pd.read_sql_query("SELECT w.date, cs.state_case FROM weather AS w LEFT JOIN case_specifics AS cs ON cs.year || '-' || cs.month || '-' || cs.day = w.date", conn)
testquery

,date,state_case
0,2022-10-01,131251
1,2022-10-02,131267
2,2022-10-03,None
3,2022-10-04,131314
4,2022-10-05,None
...,...,...
378,2023-09-27,None
379,2023-09-28,None
380,2023-09-29,None
381,2023-09-30,None


In [ ]:
#cursor.executescript("DROP VIEW crashes_by_precip_by_day")
#conn.commit()

To do:
create the views to draw conclusions between vehicle crashes, weather conditions, and ozone levels
analyze database schema (look at how to make relations)
implement the running of this program on a batch pipeline with tooling
add commentary to ETL_Functions and pipeline
add README file explaining the project and parts of it
move api key to separate txt file and import it in